<h2>Update Database at 8 AM</h2>

In [1]:
from pandas_datareader import data as pdr
# pip install pandas-datareader
import quandl
# pip install quandl
from googlefinance import getQuotes
# pip install googlefinance
import json
import joblib

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
cred = credentials.Certificate('./one-day-stock-afb526f07b58.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://one-day-stock-default-rtdb.firebaseio.com'
})

In [3]:
def QueryStockIndex(name, startDate, endDate):
    data = pdr.get_data_yahoo(name, start=startDate, end=endDate)
    return data

In [13]:
def QueryOilPrice(startDate, endDate):
    quandl.ApiConfig.api_key = "RgFSGT7KyM8eqQzDgers"
    OilPrice = quandl.get("OPEC/ORB", start_date=startDate,end_date=endDate)
    return OilPrice

In [14]:
import datetime

dateData = datetime.datetime(2021, 3, 22)
if (dateData.strftime("%w") == '0'):
    dateStart = dateData - datetime.timedelta(days=2)
    dateEnd = dateData - datetime.timedelta(days=2)
elif (dateData.strftime("%w") == '6'):
    dateStart = dateData - datetime.timedelta(days=1)
    dateEnd = dateData - datetime.timedelta(days=1)
else :
    dateStart = dateData
    dateEnd = dateData

print(dateStart)

2021-03-22 00:00:00


In [15]:
SET = pdr.DataReader('^SET', data_source='stooq')
for index, row in SET.iterrows():
    if (str(index)[:10] == str(dateStart)[:10]):
        temp = {
            'Open' : row.Open,
            'Close' : row.Close,
            'High' : row.High,
            'Low' : row.Low
        }
ref = db.reference('/SET')
ref.child(str(dateData)[:10]).update(temp)
print('Update SET = '+ str(temp))

Update SET = {'Open': 1563.56, 'Close': 1566.36, 'High': 1571.74, 'Low': 1555.38}


In [16]:
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
for index, row in OPEC.iterrows():
    temp = {
        'value' : row.Value
    }
ref = db.reference('/OPEC')
ref.child(str(dateData)[:10]).update(temp)
print('Update OPEC = '+ str(temp))

Update OPEC = {'value': 62.47}


In [17]:
modelPTT = joblib.load('../SVM/model/best/PTT.joblib')
PTT = QueryStockIndex("PTT.BK", dateStart, dateEnd)
# OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=1), dateEnd - datetime.timedelta(days=1))
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
result = modelPTT.predict([[float(PTT.High), float(PTT.Low), float(PTT.Open), float(PTT.Close), float(PTT.Volume), float(OPEC.Value)]])
print(dateStart)
if (dateData.strftime("%w") == '0') or (dateData.strftime("%w") == '6'):
    tempVolume = 0
else :
    tempVolume = int(PTT.Volume)
temp = {
    'close': round(float(PTT.Close),2),
    'high': round(float(PTT.High),2),
    'low': round(float(PTT.Low),2),
    'open': round(float(PTT.Open),2),
    'predict': round(float(result[0]),2),
    'volume': tempVolume

}
ref = db.reference('/PTT')
ref.child(str(dateData)[:10]).update(temp)
print('Update PTT = '+ str(temp))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVR from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
2021-03-22 00:00:00
Update PTT = {'close': 40.5, 'high': 40.75, 'low': 40.0, 'open': 40.5, 'predict': 40.49, 'volume': 34058900}


In [18]:
modelBCP = joblib.load('../SVM/model/best/BCP.joblib')
BCP = QueryStockIndex("BCP.BK", dateStart, dateEnd)
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
result = modelBCP.predict([[float(BCP.High), float(BCP.Low), float(BCP.Open), float(BCP.Close), int(BCP.Volume), float(OPEC.Value)]])
if (dateData.strftime("%w") == '0') or (dateData.strftime("%w") == '6'):
    tempVolume = 0
else :
    tempVolume = int(BCP.Volume)
temp = {
    'close': round(float(BCP.Close),2),
    'high': round(float(BCP.High),2),
    'low': round(float(BCP.Low),2),
    'open': round(float(BCP.Open),2),
    'predict': round(float(result[0]),2),
    'volume': tempVolume

}
ref = db.reference('/BCP')
ref.child(str(dateData)[:10]).update(temp)
print('Update BCP = '+ str(temp))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVR from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Update BCP = {'close': 26.75, 'high': 27.5, 'low': 26.5, 'open': 27.5, 'predict': 26.9, 'volume': 5345200}


In [19]:
modelIRPC = joblib.load('../SVM/model/best/IRPC.joblib')
IRPC = QueryStockIndex("IRPC.BK", dateStart, dateEnd)
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
result = modelIRPC.predict([[float(IRPC.High), float(IRPC.Low), float(IRPC.Open), float(IRPC.Close), int(IRPC.Volume), float(OPEC.Value)]])
if (dateData.strftime("%w") == '0') or (dateData.strftime("%w") == '6'):
    tempVolume = 0
else :
    tempVolume = int(IRPC.Volume)
temp = {
    'close': round(float(IRPC.Close),2),
    'high': round(float(IRPC.High),2),
    'low': round(float(IRPC.Low),2),
    'open': round(float(IRPC.Open),2),
    'predict': round(float(result[0]),2),
    'volume': tempVolume

}
ref = db.reference('/IRPC')
ref.child(str(dateData)[:10]).update(temp)
print('Update IRPC = '+ str(temp))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVR from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Update IRPC = {'close': 3.82, 'high': 3.86, 'low': 3.8, 'open': 3.84, 'predict': 3.81, 'volume': 134783800}


In [20]:
modelSUSCO = joblib.load('../SVM/model/best/SUSCO.joblib')
SUSCO = QueryStockIndex("SUSCO.BK", dateStart, dateEnd)
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
result = modelSUSCO.predict([[float(SUSCO.High), float(SUSCO.Low), float(SUSCO.Open), float(SUSCO.Close), int(SUSCO.Volume), float(OPEC.Value)]])
if (dateData.strftime("%w") == '0') or (dateData.strftime("%w") == '6'):
    tempVolume = 0
else :
    tempVolume = int(SUSCO.Volume)
temp = {
    'close': round(float(SUSCO.Close),2),
    'high': round(float(SUSCO.High),2),
    'low': round(float(SUSCO.Low),2),
    'open': round(float(SUSCO.Open),2),
    'predict': round(float(result[0]),2),
    'volume': tempVolume

}
ref = db.reference('/SUSCO')
ref.child(str(dateData)[:10]).update(temp)
print('Update SUSCO = '+ str(temp))

Update SUSCO = {'close': 3.5, 'high': 3.5, 'low': 3.42, 'open': 3.46, 'predict': 3.49, 'volume': 3076400}


In [21]:
modelTOP = joblib.load('../SVM/model/best/TOP.joblib')
TOP = QueryStockIndex("TOP.BK", dateStart, dateEnd)
x = True
a = 1
while(x):
    try :
        OPEC = QueryOilPrice(dateStart - datetime.timedelta(days=a), dateEnd - datetime.timedelta(days=a))
        float(OPEC.Value)
        x = False
    except :
        a += 1
result = modelTOP.predict([[float(TOP.High), float(TOP.Low), float(TOP.Open), float(TOP.Close), int(TOP.Volume), float(OPEC.Value)]])
if (dateData.strftime("%w") == '0') or (dateData.strftime("%w") == '6'):
    tempVolume = 0
else :
    tempVolume = int(TOP.Volume)
temp = {
    'close': round(float(TOP.Close),2),
    'high': round(float(TOP.High),2),
    'low': round(float(TOP.Low),2),
    'open': round(float(TOP.Open),2),
    'predict': round(float(result[0]),2),
    'volume': tempVolume

}
ref = db.reference('/TOP')
ref.child(str(dateData)[:10]).update(temp)
print('Update TOP = '+ str(temp))

Update TOP = {'close': 62.0, 'high': 62.5, 'low': 60.75, 'open': 61.0, 'predict': 61.9, 'volume': 4888900}
